In [7]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
cd C:/Users/syedd/Documents/mine/DCLC3

C:\Users\syedd\Documents\mine\DCLC3


In [20]:
import cv2
import numpy as np
import fiftyone as fo
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

In [35]:
images = glob("data/dataset/training_patches/*.png")
# images  = [i.replace("New_Pred_Masks" , "image_patch") for i in pred_masks]
masks= [i.replace("training_patches", "training_noisy_labels") for i in images]
pred_masks=[i.replace("training_patches","dlv3+_res101_concat_aug_epoch_004.ckpt") for i in images]
len(images) , len(masks) , len(pred_masks)
# data/dataset/training_noisy_labels

(5000, 5000, 5000)

In [37]:
# mask = cv2.imread(pred_masks[i], cv2.IMREAD_GRAYSCALE)
# plt.imshow(mask)
# data/dataset/training_noisy_labels/0_10_2_1044_1044.png

In [38]:

samples = []

if "seg_exp" in fo.list_datasets():
    fo.delete_dataset("seg_exp")  
    
for i in tqdm(range(len(images))):
    sample = fo.Sample(filepath=images[i])
    sample["noisy_labels"] = fo.Segmentation(mask=cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE))
    sample["pred_masks"] = fo.Segmentation(mask=cv2.imread(pred_masks[i], cv2.IMREAD_GRAYSCALE))
    samples.append(sample)

dataset = fo.Dataset("seg_exp")
dataset.add_samples(samples)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:32<00:00, 154.91it/s]

   0% ||--------------|    1/5000 [29.4ms elapsed, 2.4m remaining, 34.0 samples/s] 

 100% |███████████████| 5000/5000 [9.0s elapsed, 0s remaining, 586.3 samples/s]      


['677d361f7beac99cab79982d',
 '677d361f7beac99cab79982e',
 '677d361f7beac99cab79982f',
 '677d361f7beac99cab799830',
 '677d361f7beac99cab799831',
 '677d361f7beac99cab799832',
 '677d361f7beac99cab799833',
 '677d361f7beac99cab799834',
 '677d361f7beac99cab799835',
 '677d361f7beac99cab799836',
 '677d361f7beac99cab799837',
 '677d361f7beac99cab799838',
 '677d361f7beac99cab799839',
 '677d361f7beac99cab79983a',
 '677d361f7beac99cab79983b',
 '677d361f7beac99cab79983c',
 '677d361f7beac99cab79983d',
 '677d361f7beac99cab79983e',
 '677d361f7beac99cab79983f',
 '677d361f7beac99cab799840',
 '677d361f7beac99cab799841',
 '677d361f7beac99cab799842',
 '677d361f7beac99cab799843',
 '677d361f7beac99cab799844',
 '677d361f7beac99cab799845',
 '677d361f7beac99cab799846',
 '677d361f7beac99cab799847',
 '677d361f7beac99cab799848',
 '677d361f7beac99cab799849',
 '677d361f7beac99cab79984a',
 '677d361f7beac99cab79984b',
 '677d361f7beac99cab79984c',
 '677d361f7beac99cab79984d',
 '677d361f7beac99cab79984e',
 '677d361f7bea

In [42]:
# from glob import glob
# import fiftyone as fo
# import cv2
# from tqdm import tqdm

# samples = []

# if "seg_exp" in fo.list_datasets():
#     fo.delete_dataset("seg_exp")  

# for i in tqdm(range(len(images))):
#     mask = cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE)
#     pred_mask = cv2.imread(pred_masks[i], cv2.IMREAD_GRAYSCALE)
    
#     if mask is None:
#         print(f"Error reading mask: {masks[i]}")
#         continue
#     if pred_mask is None:
#         print(f"Error reading predicted mask: {pred_masks[i]}")
#         continue

#     sample = fo.Sample(filepath=images[i])
#     sample["noisy_labels"] = fo.Segmentation(mask=mask)
#     sample["pred_masks"] = fo.Segmentation(mask=pred_mask)
#     samples.append(sample)

# dataset = fo.Dataset("seg_exp")
# dataset.add_samples(samples)

In [39]:
import fiftyone as fo
import pandas as pd

csv_file = "Submission Csv/dlv3+_res101_concat_aug_raw.csv"
iou_df = pd.read_csv(csv_file ,index_col= False)
iou_df["image_path"] = "C:\\Users\\syedd\\Documents\\mine\\DCLC3\\data\\dataset\\training_patches\\" + iou_df["image_id"]
iou_df.head()

# Mask Path'] = iou_df['Mask Path'].apply(lambda path: os.path.normpath(path))


,iou,image_id,image_path
0,0.879055,1_7_87_209_418.png,C:\Users\syedd\Documents\mine\DCLC3\data\datas...
1,0.869233,0_40_60_1044_1044.png,C:\Users\syedd\Documents\mine\DCLC3\data\datas...
2,0.863141,0_40_60_1044_209.png,C:\Users\syedd\Documents\mine\DCLC3\data\datas...
3,0.858000,1_13_89_836_209.png,C:\Users\syedd\Documents\mine\DCLC3\data\datas...
4,0.852218,2_26_34_0_1044.png,C:\Users\syedd\Documents\mine\DCLC3\data\datas...


In [40]:
dataset = fo.load_dataset("seg_exp")
for index, row in tqdm(iou_df.iterrows()):
    sample = dataset.match({"filepath": row['image_path']}).first()

    if sample:
        sample["iou"] = row["iou"]
        sample.save()

5000it [00:36, 138.41it/s]


In [41]:
view = dataset.sort_by("iou", reverse=True)
session = fo.launch_app(view , port = 5051)


In [45]:
ds = session.view

In [48]:
for sample in tqdm(ds):
    break

  0%|                                                                                          | 0/126 [00:00<?, ?it/s]


In [49]:
sample["filepath"]

'C:\\Users\\syedd\\Documents\\mine\\DCLC3\\data\\dataset\\training_patches\\0_40_60_1044_1044.png'

In [136]:
file_paths = [sample.filepath for sample in view.skip(500).limit(500)]

In [43]:
top126_dataset = fo.load_dataset("Top126")

DatasetNotFoundError: Dataset Top126 not found

In [7]:
voxel --port 5051

SyntaxError: invalid syntax (3312517765.py, line 1)